In [2]:
!pip install snscrape pymongo streamlit




In [3]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pymongo
import streamlit as st



In [4]:
def scrape_tweets(keyword, start_date, end_date, limit):
    tweets = []
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f'{keyword} since:{start_date} until:{end_date}').get_items()):
        if i >= limit:
            break
        tweet_dict = {}
        tweet_dict['date'] = tweet.date.strftime('%Y-%m-%d %H:%M:%S')
        tweet_dict['id'] = tweet.id
        tweet_dict['url'] = tweet.url
        tweet_dict['content'] = tweet.content
        tweet_dict['user'] = tweet.user.username
        tweet_dict['reply_count'] = tweet.replyCount
        tweet_dict['retweet_count'] = tweet.retweetCount
        tweet_dict['language'] = tweet.lang
        tweet_dict['source'] = tweet.sourceLabel
        tweet_dict['like_count'] = tweet.likeCount
        tweets.append(tweet_dict)
    return tweets


In [5]:
def store_tweets(keyword, start_date, end_date, limit):
    tweets = scrape_tweets(keyword, start_date, end_date, limit)
    db_client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = db_client['twitter_db']
    collection = db[keyword]
    collection.insert_many(tweets)


In [6]:
st.title('Twitter Scraper')
keyword = st.text_input('Enter keyword or hashtag:')
start_date = st.date_input('Start date:')
end_date = st.date_input('End date:')
limit = st.slider('Limit:', min_value=10, max_value=1000, step=10, value=100)
if st.button('Scrape'):
    store_tweets(keyword, start_date, end_date, limit)
    st.success('Scraping successful')
if st.button('Download CSV'):
    db_client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = db_client['twitter_db']
    collection = db[keyword]
    tweets = collection.find()
    df = pd.DataFrame(list(tweets))
    st.download_button('Download CSV', df.to_csv(), mime='text/csv')
if st.button('Download JSON'):
    db_client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = db_client['twitter_db']
    collection = db[keyword]
    tweets = collection.find()
    data = []
    for tweet in tweets:
        tweet.pop('_id')
        data.append(tweet)
    st.download_button('Download JSON', pd.Series(data).to_json(), mime='application/json')


2023-04-06 10:57:03.888 
  command:

    streamlit run C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
